In [ ]:
HW4 - Fake News Detection
本次作業將針對假新聞作分析，預測一則新聞是否reliable (繳交期限：12/10 23:55)
1: fake
0: true
請分別利用GBDT、LightGBM、xgboost對"train.csv"的資料建模，並用"test.csv"進行測試
註："test.csv"的label在"sample_submission.csv"裡面
(資料來源：https://www.kaggle.com/c/fakenewskdd2020/data)

作業流程：

1. 資料前處理
 a. 讀取"train.csv"與"test.csv"並利用分割符號切割、建立train&test之DataFrame
註：分割符號為tab(\t)

 b. 去除停頓詞stop words 
可參考：
sklearn.feature_extraction.text.CountVectorizer
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

自訂stop words
https://stackoverflow.com/questions/52712254/how-to-eliminate-stop-words-only-using-scikit-learn

 c. 文字探勘前處理，將文字轉換成向量，像是常見的方法 tf-idf、word2vec...等

可參考：
sklearn.feature_extraction.text.TfidfVectorizer
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text

Word2vec
https://radimrehurek.com/gensim/models/word2vec.html

2. 建模：分別使用以下三種模型
xgboost
https://xgboost.readthedocs.io/en/latest/python/python_intro.html#install-xgboost

GBDT

LightGBM
https://machinelearningmastery.com/gradient-boosting-with-scikit-learn-xgboost-lightgbm-and-catboost/

3. 評估模型
利用"test.csv"的資料對2.所建立的模型進行測試，並計算Accuracy、Precision、Recall、F-measure

請將以下檔案壓縮成 HW4_學號_姓名.zip：

1. 討論報告PDF (含截圖跟簡短說明、討論等等)，需提及：
文字探勘前處理的方法
GBDT、LightGBM、xgboost 模型之結果比較

2. Python程式碼

In [27]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv", sep='\t', encoding='utf-8')
test = pd.read_csv("test.csv", sep='\t', encoding='utf-8')
test_label = pd.read_csv("sample_submission.csv", sep='\t', encoding='utf-8')
train.head()

# type(train) : pandas.core.frame.DataFrame
# train.columns : Index(['text', 'label'], dtype='object')
# print( len(train['text'])): 4987
# print(len(test['text'])): 1247

,text,label
0,Get the latest from TODAY Sign up for our news...,1
1,2d Conan On The Funeral Trump Will Be Invited...,1
2,It’s safe to say that Instagram Stories has fa...,0
3,Much like a certain Amazon goddess with a lass...,0
4,At a time when the perfect outfit is just one ...,0


In [5]:
test_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id,label  1247 non-null   object
dtypes: object(1)
memory usage: 9.9+ KB


In [10]:
# 把 sample_submission.csv 中的 id 與 label切開，再塞入 test

label=[]
for i in range(len(test_label["id,label"])):
    temp_label = test_label["id,label"][i][-1]
    label.append(int(temp_label))
    
test_label["label"]=label

test["label"]= test_label["label"]
test

,id,text,label
0,2,The 2017 Teen Choice Awards ceremony was held ...,1
1,3,"The concert, part of “The Joshua Tree Tour,” w...",1
2,4,Selena Gomez refuses to talk to her mother abo...,0
3,5,This is worse than a lump of coal in your stoc...,0
4,6,Luann De Lesseps is going to rehab after her a...,0
...,...,...,...
1242,1244,Get the latest from TODAY Sign up for our news...,0
1243,1245,Jaden Smith claims that the Four Seasons Hotel...,0
1244,1246,Overview (3) Mini Bio (1) Faith Hill was bor...,1
1245,1247,CLOSE Aaron Paul dishes on 'The Path' Aaron P...,1


In [10]:
train["text"][0]

'Get the latest from TODAY Sign up for our newsletter  No one ever truly gets over losing a loved one, and Blake Shelton is no exception. He was just 14 when his older brother Richie died on Nov. 13, 1990. And, as Shelton noted in a tweet Monday, "It changed my life forever."  Richie was 24 when he died in a car accident in the Sheltons\' home state of Oklahoma. Two years ago, Shelton sent out a message for the 25th anniversary of his loss:  Richie, who was Blake\'s half-brother (they shared a mother), was a passenger in a car that collided with a school bus in Ada, south of Oklahoma City.  Richie, driver Redena McManus and a 3-year-old boy, Christopher McManus, all died during or shortly after the collision, while the bus driver and passengers were uninjured, according to police reports.  The accident has clearly remained with Blake, who told 60 Minutes in 2014, "I remember picking up the phone to call him a week after he was dead, to tell him something. I was picking up the phone to 

In [209]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4987 entries, 0 to 4986
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4987 non-null   object
 1   label   4987 non-null   object
dtypes: object(2)
memory usage: 78.0+ KB


In [210]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1247 non-null   int64 
 1   text    1247 non-null   object
 2   label   1247 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 29.4+ KB


In [11]:
# train["label"].value_counts()
# 0        2972
# 1        2014
# label       1
# Name: label, dtype: int64
# 有個 label夾在0.1裡面，抓出來刪掉row

# a=0
# for i in train["label"]:
#     if i!="label":
#         a+=1
#     if i=="label":
#         break
# a=1615
train = train.drop([1615])

In [12]:
train["label"].value_counts()

0    2972
1    2014
Name: label, dtype: int64

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# 引用 NLTK 的 stop words list
stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# 將文字轉成向量，同時 去除停頓詞stop words 
vectorizer = CountVectorizer(stop_words=stop_words, max_features=2500)

train_vector = vectorizer.fit_transform(train["text"]).toarray()
test_vector = vectorizer.fit_transform(test["text"]).toarray()

# print(vectorizer.get_feature_names())


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

transform = TfidfTransformer()    # 使用TF-IDF（詞頻、逆文檔頻率）應用於稀疏矩陣
Y = transform.fit_transform(X)    # 使用上面CountVectorizer處理後的 X 數據
print(Y.toarray())                # 輸出轉換爲tf-idf後的 Y 矩陣，同樣直接打印 Y 輸出每個數據的位置
print(vectorizer.get_feature_names())    # 打印特徵名

In [8]:
len(stop_words)

179

In [14]:
print(train_vector)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [26]:
a=0
for i in train_vector[0]:
    if i > 1:
        print(i)
        a+=1
        if a>5:
            break

2
6
3
2
2
2


In [15]:
train_X = train_vector
train_y = train["label"].values
test_X = test_vector
test_y = test["label"].values

In [241]:
"""
xgboost
"""
import xgboost as xgb

# read in data
dtrain = xgb.DMatrix(train_X , label=train_y)
dtest = xgb.DMatrix(test_X , label=test_y)
# specify parameters via map
param = {'max_depth': 25, 'eta': 1, 'objective': 'reg:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

num_round = 5 # 2:電腦跑10秒、5:電腦跑30秒外加風扇全開，決定跑到5就好，不要虐待電腦。recall從0.1升到0.2已經夠了。
bst = xgb.train(param, dtrain, num_round)

#After training, the model can be saved.
bst.save_model('0001.model')

#A saved model can be loaded as follows:
bst = xgb.Booster({'nthread': 4})  # init model
bst.load_model('0001.model')  # load data

# make prediction
#preds = bst.predict(dtest)

In [242]:
# 預測數據集 
y_pred = bst.predict(dtest)

In [ ]:
import matplotlib
#畫出 XGB的樹
xgb.to_graphviz(bst)

In [243]:
# 計算Precision, Recall, Accuracy
# II. 函式計算
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision = precision_score(test_y,y_pred.round())
recall = recall_score(test_y,y_pred.round())
accuracy = accuracy_score(test_y,y_pred.round())
f1_measure = f1_score(test_y,y_pred.round())
print("XGboost precision=",precision)
print("XGboost recall=",recall)
print("XGboost accuracy=",accuracy)
print("XGboost f1_score=",f1_measure)


XGboost precision= 0.47985347985347987
XGboost recall= 0.21231766612641814
XGboost accuracy= 0.4963913392141139
XGboost f1_score= 0.2943820224719101


In [19]:
train_y=[]
for i in train["label"]:
    a = int(i)
    train_y.append(a)

In [20]:
train_X = train_vector
train_y = train_y
test_X = test_vector
test_y = test["label"].values

In [21]:
"""
GBDT
"""
# from sklearn.experimental import enable_hist_gradient_boosting
# from sklearn import ensemble

# model = ensemble.HistGradientBoostingClassifier()
# model.fit(train_X, train_y)
# y_pred_GBDT=model.predict(test_X)
# 跑超過4分鐘，終於沒有error，但是整個chrome跳出"記憶體滿載"的提示後強制關閉

# 跑超過1小時，只好先中止
# from GBDT.gbdt import GradientBoostingBinaryClassifier
# model = GradientBoostingBinaryClassifier(learning_rate=0.1, max_depth=10)
# model.fit(train_X, train_y)
# y_pred_GBDT=model.predict(test_X)

# 為什麼跑5分鐘都跑不出來東西RRR，又不是多大的模型
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)
clf.fit(train_X, train_y)


GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [22]:
y_pred_GBDT = clf.predict(test_X)

In [23]:
# 計算Precision, Recall, Accuracy
# II. 函式計算
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision = precision_score(test_y,y_pred_GBDT.round())
recall = recall_score(test_y,y_pred_GBDT.round())
accuracy = accuracy_score(test_y,y_pred_GBDT.round())
f1_measure = f1_score(test_y,y_pred_GBDT.round())
print("GBDT precision=",precision)
print("GBDT recall=",recall)
print("GBDT accuracy=",accuracy)
print("GBDT f1_score=",f1_measure)

GBDT precision= 0.5158924205378973
GBDT recall= 0.3419773095623987
GBDT accuracy= 0.5156375300721732
GBDT f1_score= 0.41130604288499023


In [24]:
# 計算Precision, Recall, Accuracy
# II. 函式計算
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision = precision_score(test_y,y_pred_GBDT)
recall = recall_score(test_y,y_pred_GBDT)
accuracy = accuracy_score(test_y,y_pred_GBDT)
f1_measure = f1_score(test_y,y_pred_GBDT)
print("GBDT precision=",precision)
print("GBDT recall=",recall)
print("GBDT accuracy=",accuracy)
print("GBDT f1_score=",f1_measure)

GBDT precision= 0.5158924205378973
GBDT recall= 0.3419773095623987
GBDT accuracy= 0.5156375300721732
GBDT f1_score= 0.41130604288499023


In [193]:
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)

clf.predict(X_test[:2])

clf.score(X_test, y_test)

0.88

In [244]:
len(X_train)

75

In [246]:
X_train

array([[-0.65240858,  0.49374178,  1.30184623, ..., -1.30819171,
        -1.04525337, -0.11054066],
       [ 0.35178011, -0.47003288, -0.37914756, ..., -2.38076394,
        -0.11048941, -1.55042935],
       [-1.58249448, -1.42279491, -0.56430103, ...,  1.26661394,
        -1.31771734,  1.61805427],
       ...,
       [-0.96050438, -2.28862004,  1.02943883, ..., -0.79347019,
         1.12859406, -0.27567053],
       [ 0.91017891,  0.78632796,  0.06326199, ...,  0.42234144,
        -0.46359597, -0.01702041],
       [-0.87916063, -1.63880731, -0.30769128, ..., -0.6054158 ,
         1.57886519,  0.73165893]])

In [245]:
len(y_train)

75

In [247]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0])

In [199]:
y_test

array([1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0])

In [264]:
"""
Light GBM
"""

import lightgbm as lgb 

train_data = lgb.Dataset(train_X , label=train_y)

param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'

num_round = 10
bst = lgb.train(param, train_data, num_round)

# After training, the model can be saved:
bst.save_model('model.txt')

# A saved model can be loaded:
bst = lgb.Booster(model_file='model.txt')  # init model

ypred_LightGBM = bst.predict(test_X, predict_disable_shape_check="true")

[LightGBM] [Info] Number of positive: 2014, number of negative: 2972
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.067092 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 42462
[LightGBM] [Info] Number of data points in the train set: 4986, number of used features: 7978
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.403931 -> initscore=-0.389112
[LightGBM] [Info] Start training from score -0.389112


In [267]:
# 計算Precision, Recall, Accuracy
# II. 函式計算
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision_LightGBM = precision_score(test_y,ypred_LightGBM.round())
recall_LightGBM = recall_score(test_y,ypred_LightGBM.round())
accuracy_LightGBM = accuracy_score(test_y,ypred_LightGBM.round())
f1_measure_LightGBM = f1_score(test_y,ypred_LightGBM.round())
print("Light GBM precision=",precision_LightGBM)
print("Light GBM recall=",recall_LightGBM)
print("Light GBM accuracy=",accuracy_LightGBM)
print("Light GBM f1_score=",f1_measure_LightGBM)

Light GBM precision= 0.49390243902439024
Light GBM recall= 0.1312803889789303
Light GBM accuracy= 0.5036086607858862
Light GBM f1_score= 0.20742637644046094
